In [1]:
import boto3
import pandas as pd
s3 = boto3.client('s3')

with open('balanced_reviews.csv', 'wb') as data:
    s3.download_fileobj('dlfinalproject', 'balanced_reviews.csv', data)
df = pd.read_csv('balanced_reviews.csv')

In [2]:
s3 = boto3.resource('s3')
s3.meta.client.upload_file('my project_LSTM-CNN.ipynb', 'dlfinalproject', 'my project.ipynb')

In [3]:
df.head()

,Unnamed: 0,business_id,cool,date,funny,review_id,stars,text,useful,user_id,fun_bin
0,17,cHdJXLlKNWixBXpDwEGb_A,1,2015-04-01 16:30:00,7,6BnQwlxRn7ZuWdzninM9sQ,3,I love chinese food and I love mexican food. W...,1,JSrP-dUmLlwZiI7Dp3PQ2A,1
1,21,Mem13A3C202RzT53npn4NA,9,2017-05-13 10:41:43,6,IPw8yWiyqnfBzzWmypUHgg,5,If you are looking for the best pierogies in P...,9,5JVY32_bmTBfIGpCCsnAfw,1
2,62,SU56w479vUfFHsvmvQIf7A,6,2016-07-25 03:55:20,5,E4LqIZ7DJd_R4ZHSNKx4RQ,4,So good! They didn't make it to 5 stars due to...,7,DoRCeCcJbrsM2BiAKj3trA,1
3,126,tjAeaGdxf7I4xN9M7wGJNQ,4,2014-07-13 14:32:56,5,TaoaX7MqCujFRNaJBns2Sw,5,While the prices are a bit high for a make-you...,8,x37OyP--VEFE5p-xreplYA,1
4,246,FhIeCF6QrsLaRvAeu0oEPQ,4,2013-06-24 06:42:29,5,3Qc49B7dA0ONmCxrn5iwCQ,2,OVERALL: The food isn't good (I explain below)...,13,2k8OVAPxlXHsA5X6EIoQpQ,1


In [4]:
import os
import re
import string

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

'''from nltk.tokenize import WordPunctTokenizer
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer
'''
from sklearn.feature_extraction.text import CountVectorizer

In [5]:
df_reviews = df[['funny','text', 'fun_bin']]

In [6]:
pd.set_option('display.max_colwidth', -1)
df_reviews.head()

,funny,text,fun_bin
0,7,"I love chinese food and I love mexican food. What can go wrong? A couple of things. First things first, this place is more of a ""rice bowl"" kind of place. I thought it was going to be more diverse as far as the menu goes, but its mainly rice bowls you get with different kinds of meats. The ordering was a little confusing at first, but one of the employees helped us out and I got the 2-item bowl and got the jade chicken and hengrenade chicken with all rice(jerk). I also ordered a jade chicken quesadilla on the side.\n\nI'm gonna admit, this place looks kinda dirty. I don't think Arizona uses those health department letter grade system like California does, but if I were to just judge by how it looked inside, i'd give it a ""C"" grade lol. We waited for about 15 minutes or so and finally got our food. We took it to go and ate at our hotel room. \n\nMmmm... the food was just alright. The jade chicken was nothing special. It tasted like any generic chinese fast food orange chicken/sesame chicken variant. The hengrenade chicken, although was the less spicier version of the jerk chicken, was still pretty spicy for me. Just be warned the jerk chicken is super spicy. If you aren't sure, ask for a sample at the restaurant before ordering, but it was way too spicy for me. \n\nThe jade chicken quesadilla was decent, but nothing special. Just imagine orange chicken in between a tortilla and cheese. A friend of mine ordered a jade chicken burrito and we were confused when we pulled it out of the bag because it was literally the size of Mcdonald's apple pie. If you order the burrito, be warned that it's a burrito for gnomes and smurfs, but he said it was tasty. \n\nThey provide a snicker doodle sugar cookie for each meal and it was decent, again nothing special. \n\nNot gonna lie, the next day my stomach felt like a little mexican dude and chinese dude were wrestling and throwing molotov cocktails inside. I used the bathroom like 5 times. I don't recommend eating this place if you have a lot to do the next day.",1
1,6,"If you are looking for the best pierogies in Pittsburgh, this is your place. There are a few small tables outside but most of the business is carry out. Pierogies Plus wins Best Pierogies every year. Why? Because the owner is from Poland and she is making the real deal pierogies. The best part is that they are hand pinched by a group of older Polish and Hungarian women. \nThe biggest seller is potato and cheese but they sell many flavors. They are like plump pillows of softness. You can buy them buy the dozen. You can get them cold to take home and freeze or warm and ready to eat. The warm ones are served with butter and onions. It's definitely a comfort food. The best part is that they ship internationally. Yes, they are that good.",1
2,5,"So good! They didn't make it to 5 stars due to the prices are a bit high for the amount of food and the location is a bit unsavory. \nThe decor and atmosphere was surprisingly nice, from the outside I expected to be more run down inside. The staff was very nice. We were surprised how empty the dining room was for a Friday evening.\nWe got Vegetable Samosas to start then ordered Chicken Tikka Masala, Lamb Rogan Josh, rice and plain Naan. Our only complaint was the lamb could've been more tender but everything was flavorful and delicious. \nI would definitely go again if given the chance.",1
3,5,"While the prices are a bit high for a make-your-own pizza, the taste makes up for it. I love going to Seventh Street market, sitting Not Just Coffee and having a drink while waiting for delicious fresh made pizza from Pure. I've taken this to go as well as eaten inside the market, and I can say that the pizza doesn't do well reheated. So try to eat it fresh while there if possible.\n\nIf one of their specialty pizzas sounds good to you, go for it, as those are definitely a better deal for the amount of toppings you get for the money. I wanted what I wanted, though, so I en

In [7]:
import pandas as pd
import numpy as np
#import nltk
#from nltk.corpus import stopwords
#from nltk.stem import SnowballStemmer

from string import punctuation


import numpy as np 
import pandas as pd 
import os
import spacy
import string
import re
import numpy as np
from spacy.symbols import ORTH
from collections import Counter

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence 
#from spacymoji import Emoji

In [8]:
#Run two commands below and then restart the kernel
#!pip install --upgrade pip

In [9]:
#!pip install -U spacy[cuda92]

Requirement already up-to-date: spacy[cuda92] in /home/ec2-user/anaconda3/envs/pytorch_p36/lib/python3.6/site-packages (2.1.4)


In [10]:
#!python3 -m spacy download en_core_web_sm

✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_sm')


In [8]:
import spacy

spacy.prefer_gpu()
spacy.load("en_core_web_sm")

In [12]:
import pandas as pd
import numpy as np
#import nltk
#from nltk.corpus import stopwords
#from nltk.stem import SnowballStemmer

from string import punctuation


import numpy as np 
import pandas as pd 
import os
import spacy
import string
import re
import numpy as np
from spacy.symbols import ORTH
from collections import Counter
from spacymoji import Emoji

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence 

In [10]:
#!pip install spacymoji

In [13]:

re_br = re.compile(r'<\s*br\s*/?>', re.IGNORECASE)
def sub_br(x): return re_br.sub("\n", x)

#nlp = spacy.load("en")
#spacy.load('en')
spacy_stopwords = spacy.lang.en.stop_words.STOP_WORDS



def clean_text(text):
    ''' Pre process and convert texts to a list of words '''
    text = str(text)
    text = text.lower()

    # Clean the text
   # text = re.sub(r"[^A-Za-z0-9^,!.\/'+-=\(\)]", " ", text) # keep punctuatuin, numnbers and letters
    text = re.sub(r"what's", "what is ", text)
    text = re.sub(r"\'s", " ", text)
    text = re.sub(r"\'ve", " have ", text)
    text = re.sub(r"can't", "cannot ", text)
    text = re.sub(r"n't", " not ", text)
    text = re.sub(r"i'm", "i am ", text)
    text = re.sub(r"\'re", " are ", text)
    text = re.sub(r"\'d", " would ", text)
    text = re.sub(r"\'ll", " will ", text)
    text = re.sub(r",", " ", text)
    text = re.sub(r"\.", " . ", text) #Add space to the dot
    text = re.sub(r"!", " ! ", text) #Add space to the exclamation sign
    text = re.sub(r":", " :", text) #Add space before : sign
    text = re.sub(r";", " ;", text) #Add space before ; sign
    text = re.sub(r"\/", " ", text)
    text = re.sub(r"\^", " ^ ", text)
    text = re.sub(r"\+", " + ", text)
    text = re.sub(r"\-", " - ", text)
    text = re.sub(r"\=", " = ", text)
    text = re.sub(r"'", " ", text)
    #text = re.sub(r"(\d+)(k)", r"\g<1>000", text)
    #text = re.sub(r":", " : ", text)
    text = re.sub(r" e g ", " eg ", text)
    text = re.sub(r" b g ", " bg ", text)
    text = re.sub(r" u s ", " american ", text)
    text = re.sub(r"\0s", "0", text)
    text = re.sub(r" 9 11 ", "911", text)
    text = re.sub(r"e - mail", "email", text)
    text = re.sub(r"j k", "jk", text)
    text = re.sub(r"\s{2,}", " ", text)
    # find emojis
    emoji_list = []
    '''
    for word in text.split():
        if any(char in emoji.UNICODE_EMOJI for char in word):
            emoji_list.append(word)
    emoji_list'''
    #text = text.split()

    return text

my_tok = spacy.load('en_core_web_sm')
emoji = Emoji(my_tok)
my_tok.add_pipe(emoji, first=True)
def spacy_tok(x): return [tok.text for tok in my_tok.tokenizer(clean_text(x))]

def remove_stop_words(tokens): return [tok for tok in tokens if tok not in spacy_stopwords]


### Building a vocabulary

In [14]:
from collections import Counter
counts = Counter()
for sent in df_reviews['text']:
    try:
        counts.update(remove_stop_words(spacy_tok(sent)))
    except:
        pass

In [18]:
#counts

### Vocabulary

In [19]:
# Vocabulary
vocab2index = {"":0, "UNK":1}
words = ["", "UNK"]
for word in counts:
    vocab2index[word] = len(words)
    words.append(word)

In [20]:
# WHat is the 99% quantile of  length of the sentence?

df_reviews['len_text'] = df_reviews['text'].apply(lambda x: len(x.split()))


/home/ec2-user/anaconda3/envs/pytorch_p36/lib/python3.6/site-packages/ipykernel/__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()


In [21]:
df_reviews['len_text'].quantile(0.95)

496.0

In [22]:
# note that spacy_tok takes a while run it just once
def encode_sentence(sent, vocab2index, N=500, padding_start=True):
    "Encoding a sentence adding padding"
    x = remove_stop_words(spacy_tok(sent))
    enc = np.zeros(N, dtype=np.int32)
    enc1 = np.array([vocab2index.get(w, vocab2index["UNK"]) for w in x])
    l = min(N, len(enc1))
    if padding_start:
        enc[:l] = enc1[:l]
    else:
        enc[N-l:] = enc1[:l]
    return enc, l

### Splitting into train and validation sets

In [23]:
from sklearn.model_selection import train_test_split
X_train, X_valid, y_train, y_valid = train_test_split(df_reviews['text'], df_reviews['fun_bin'], test_size=0.2, random_state=42)
    

In [24]:
X_train.reset_index(inplace=True, drop=True)
X_valid.reset_index(inplace=True, drop=True)

In [25]:
y_train.reset_index(inplace=True, drop=True)
y_valid.reset_index(inplace=True, drop=True)

In [26]:
y_train.shape

(120401,)

### Writing a dataset

In [27]:
class YelpDataset(Dataset):
    def __init__(self, df, y, N=400, padding_start=True):
        self.df = df
        self.X = [encode_sentence(sent, vocab2index, N, padding_start) for sent in self.df]
        self.y = y
        
    def __len__(self):
        return len(self.y)
    
    def __getitem__(self, idx):
        x, s = self.X[idx]
        return x, s, self.y[idx]


In [28]:
train_ds =  YelpDataset(X_train, y_train, padding_start=False)
valid_ds =  YelpDataset(X_valid, y_valid, padding_start=False)


In [29]:
neg=[]
i=0
for x,s,y in train_ds:
    if s <=0:
        neg.append(i)
    i+=1

neg

[118582]

In [30]:
X_train.loc[118582,]

'Neither'

In [31]:
X_train.drop(index = 118582, inplace=True)
y_train.drop(index = 118582, inplace=True)
X_train.reset_index(inplace=True, drop=True)
y_train.reset_index(inplace=True, drop=True)

In [32]:
batch_size = 1000
train_dl = DataLoader(train_ds, batch_size=batch_size, shuffle=True)
valid_dl = DataLoader(valid_ds, batch_size=batch_size)

### LSTM-CNN

In [33]:
train_ds =  YelpDataset(X_train, y_train, padding_start=False)
valid_ds =  YelpDataset(X_valid, y_valid, padding_start=False)


In [34]:
batch_size = 5
train_dl = DataLoader(train_ds, batch_size=batch_size, shuffle=True)
valid_dl = DataLoader(valid_ds, batch_size=batch_size)

In [38]:
V = vocab_size = len(words)
D = 300
print(V)

133541


In [39]:
x,s,y = next(iter(train_dl))

In [40]:
x

tensor([[   0,    0,    0,  ...,    4, 4641,    6],
        [   0,    0,    0,  ...,   87, 7472,  183],
        [   0,    0,    0,  ...,  273, 1060,  269],
        [   0,    0,    0,  ..., 2340,  139,    6],
        [   0,    0,    0,  ..., 3369,  563,    6]], dtype=torch.int32)

In [44]:
y

tensor([1, 1, 1, 1, 1])

In [41]:
# LSTM
embedding_dim = 300
hidden_dim = 50

embedding = nn.Embedding(vocab_size, embedding_dim, padding_idx=0)
lstm = nn.LSTM(embedding_dim, hidden_dim, batch_first=True)
linear = nn.Linear(hidden_dim, 1)
dropout = nn.Dropout(0.5)
        

In [45]:
x1 = embedding(x.long())

In [46]:
x1.shape

torch.Size([5, 400, 300])

In [47]:
#x = self.embeddings(x)
x2 = dropout(x1)
x2.size()
#out_pack, (ht, ct) = self.lstm(x)
#self.linear(ht[-1])

torch.Size([5, 400, 300])

In [48]:
out_pack, (ht, ct) = lstm(x2)

In [ ]:
# needs to convert x to (batch, embedding_dim, sentence_len)

In [ ]:
# needs to convert x to (batch, embedding_dim, hidden_dim)

In [57]:
out_pack.size()

torch.Size([5, 400, 50])

In [50]:
ht.size()

torch.Size([1, 5, 50])

In [51]:
ht[-1].size()

torch.Size([5, 50])

In [52]:
linear(ht[-1]).size()

torch.Size([5, 1])

In [74]:
lstm_out = out_pack.transpose(1,2) # needs to convert x to (batch, embedding_dim, sentence_len)
lstm_out.size()

torch.Size([5, 50, 400])

In [53]:
conv_3 = nn.Conv1d(in_channels=D, out_channels=100, kernel_size=3)

In [55]:
x3 = conv_3(x2)

RuntimeError: Given groups=1, weight of size 100 300 3, expected input[5, 400, 300] to have 300 channels, but got 400 channels instead

In [56]:
x3.size()

torch.Size([5, 100, 398])

In [57]:
conv_4 = nn.Conv1d(in_channels=hidden_dim, out_channels=100, kernel_size=4)
conv_5 = nn.Conv1d(in_channels=hidden_dim, out_channels=100, kernel_size=5)

In [58]:
x4 = conv_4(lstm_out)
x5 = conv_5(lstm_out)

In [59]:
print(x4.size(), x5.size())

torch.Size([5, 100, 397]) torch.Size([5, 100, 396])


In [60]:
# Combine
# 100 3-gram detectors
x3 = nn.ReLU()(x3)
x3 = nn.MaxPool1d(kernel_size = 398)(x3)
x3.size()

torch.Size([5, 100, 1])

In [61]:
x4 = nn.ReLU()(x4)
x4 = nn.MaxPool1d(kernel_size = 397)(x4)
x4.size()

torch.Size([5, 100, 1])

In [62]:
x5 = nn.ReLU()(x5)
x5 = nn.MaxPool1d(kernel_size = 396)(x5)
x5.size()

torch.Size([5, 100, 1])

In [63]:

# concatenate x3, x4, x5
out = torch.cat([x3, x4, x5], 2)
out.size()

torch.Size([5, 100, 3])

In [64]:
out = out.view(out.size(0), -1)
out.size()

torch.Size([5, 300])

In [36]:
class LSTM_CNNModel(nn.Module):
    
    def __init__(self, vocab_size, embedding_dim, hidden_dim):
        super(LSTM_CNNModel, self).__init__()

        self.embedding = nn.Embedding(vocab_size, embedding_dim, padding_idx=0)
        #LSTM layer
        self.lstm = nn.LSTM(embedding_dim, hidden_dim, batch_first=True)
        #CNN layers
        self.conv_3 = nn.Conv1d(in_channels=hidden_dim, out_channels=100, kernel_size=3)
        self.conv_4 = nn.Conv1d(in_channels=hidden_dim, out_channels=100, kernel_size=4)
        self.conv_5 = nn.Conv1d(in_channels=hidden_dim, out_channels=100, kernel_size=5)
        self.dropout = nn.Dropout(p=0.5)
        self.fc = nn.Linear(hidden_dim, 1)
        
    def forward(self, x):
        # LSTM part
        x = self.embedding(x)
        x = self.dropout(x)
        out_pack, (ht, ct) = self.lstm(x)
        lstm_out = out_pack.transpose(1,2)
        # CNN part
        x3 = F.relu(self.conv_3(lstm_out))
        x4 = F.relu(self.conv_4(lstm_out))
        x5 = F.relu(self.conv_5(lstm_out))
        x3 = nn.MaxPool1d(kernel_size = 398)(x3)
        x4 = nn.MaxPool1d(kernel_size = 397)(x4)
        x5 = nn.MaxPool1d(kernel_size = 396)(x5)
        out = torch.cat([x3, x4, x5], 2)
        out = out.view(out.size(0), -1)
        #out = self.dropout(out)
        return self.fc(out)

In [37]:
def val_metrics(m, val_dl):
    model.eval()
    correct = 0
    total = 0
    sum_loss = 0.0
    for x, s, y in val_dl:
        x = x.long().cuda()
        y = y.float().cuda().unsqueeze(1)
        y_hat = model(x)
        loss = F.binary_cross_entropy_with_logits(y_hat, y)
        y_pred = y_hat > 0
        correct += (y_pred.float() == y).float().sum()
        total += y.shape[0]
        sum_loss += loss.item()*y.shape[0]
    return sum_loss/total, correct/total

In [38]:
def train_epocs(model, epochs=10, lr=0.001):
    parameters = filter(lambda p: p.requires_grad, model.parameters())
    optimizer = torch.optim.Adam(parameters, lr=lr)
    for i in range(epochs):
        model.train()
        sum_loss = 0.0
        total = 0
        for x, s, y in train_dl:
            x = x.long().cuda()
            y = y.float().cuda()
            y_pred = model(x)
            optimizer.zero_grad()
            loss = F.binary_cross_entropy_with_logits(y_pred, y.unsqueeze(1))
            loss.backward()
            optimizer.step()
            sum_loss += loss.item()*y.shape[0]
            total += y.shape[0]
        val_loss, val_acc = val_metrics(model, val_dl)
        if i % 5 == 1:
            print("train loss %.3f val loss %.3f and val accuracy %.3f" % (sum_loss/total, val_loss, val_acc))

In [39]:
batch_size = 500
train_dl = DataLoader(train_ds, batch_size=batch_size, shuffle=True)
val_dl = DataLoader(valid_ds, batch_size=batch_size)

In [40]:
embedding_dim = 300
hidden_dim = 50

In [41]:
vocab_size = len(words)
print(vocab_size)

model = LSTM_CNNModel(vocab_size, embedding_dim, hidden_dim).cuda()

133541


In [42]:
train_epocs(model, epochs=10, lr=0.01 )

RuntimeError: size mismatch, m1: [500 x 300], m2: [50 x 1] at /opt/conda/conda-bld/pytorch_1556653099582/work/aten/src/THC/generic/THCTensorMathBlas.cu:268

In [ ]:
class LSTMV0Model(torch.nn.Module) :
    def __init__(self, vocab_size, embedding_dim, hidden_dim) :
        super(LSTMV0Model,self).__init__()
        self.hidden_dim = hidden_dim
        self.embeddings = nn.Embedding(vocab_size, embedding_dim, padding_idx=0)
        self.lstm = nn.LSTM(embedding_dim, hidden_dim, batch_first=True)
        self.linear = nn.Linear(hidden_dim, 1)
        self.dropout = nn.Dropout(0.5)
        
    def forward(self, x):
        x = self.embeddings(x)
        x = self.dropout(x)
        out_pack, (ht, ct) = self.lstm(x)
        return self.linear(ht[-1])